In [3]:
#https://github.com/flyyufelix/cnn_finetune/blob/master/vgg19.py

from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from sklearn.metrics import log_loss


def vgg19_model(img_rows, img_cols, channel=None, num_classes=None):
    """
    VGG 19 Model for Keras
    Model Schema is based on 
    https://gist.github.com/baraldilorenzo/8d096f48a1be4a2d660d
    ImageNet Pretrained Weights 
    https://drive.google.com/file/d/0Bz7KyqmuGsilZ2RVeVhKY0FyRmc/view?usp=sharing
    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
    """
  
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(img_rows, img_cols,channel)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    # Add Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    # Loads ImageNet pre-trained data
    model.load_weights('vgg19_weights.h5')

    # Truncate and replace softmax layer for transfer learning
    model.layers.pop()
    model.outputs = [model.layers[-1].output]
    model.layers[-1].outbound_nodes = []
    model.add(Dense(num_classes, activation='softmax'))

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
import os 
import numpy as np 

img_width,img_height = 224, 224
nb_train_samples =400 
nb_validation_samples = 100
epochs = 2
batch_size = 16

data_dir = os.path.join(os.getcwd(),'BlobStorage')
validation_data_dir = os.path.join(data_dir, 'validation_data')
train_data_dir = os.path.join(data_dir, 'train_data')


    
train_datagen = ImageDataGenerator( 
    rescale=1. / 255, 
    shear_range=0.2, 
    zoom_range=0.2, 
    horizontal_flip=True) 
  


f = open("fasttext/clusterCenters.txt",'r')

lines = f.readlines()

#print(lines)

for line in lines:
  
    line = line.split()
   # print(line)
    modelName = line[0]
    classesNow = line[1:]
    print(modelName)
    print(classesNow)
    train_generator = train_datagen.flow_from_directory( 
        train_data_dir, 
         classes = classesNow,
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 


    validation_generator = test_datagen.flow_from_directory( 
        validation_data_dir, 
         classes = classesNow,
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 
    n_classes = len(np.unique(train_generator.classes))
    channel = 3
    

    
    
    n_classes = len(np.unique(train_generator.classes))
    channel = 3

    model = vgg19_model( img_width,img_height, channel=3, num_classes=n_classes)


    
    model.fit_generator( 
        train_generator, 
        steps_per_epoch=nb_train_samples // batch_size, 
        epochs=epochs, 
        validation_data=validation_generator, 
        validation_steps=nb_validation_samples // batch_size)
    modelNameStr = "abc/"  + modelName +".h5"
    model.save(modelNameStr)
  

model10
['rain', 'painting', 'hand', 'balcony', 'concert', 'photograph', 'shoe', 'river', 'coffee', 'soccer']
Found 15301 images belonging to 10 classes.
Found 3966 images belonging to 10 classes.


/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API:

Epoch 1/2
25/25 [==============================] - 320s 13s/step - loss: 2.3041 - acc: 0.0825 - val_loss: 2.3035 - val_acc: 0.1354
Epoch 2/2
25/25 [==============================] - 317s 13s/step - loss: 2.3027 - acc: 0.0875 - val_loss: 2.3019 - val_acc: 0.1667
model20
['architecture', 'river', 'money', 'edge', 'car', 'screen', 'furniture', 'meadow', 'coffee', 'flower', 'dress', 'shoulder', 'parade', 'sun', 'ocean', 'airplane', 'concert', 'painting', 'football', 'fireplace']
Found 29158 images belonging to 20 classes.
Found 7507 images belonging to 20 classes.


/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API:

Epoch 1/2
25/25 [==============================] - 326s 13s/step - loss: 2.9950 - acc: 0.0700 - val_loss: 2.9944 - val_acc: 0.0625
Epoch 2/2
18/25 [====================>.........] - ETA: 1:25 - loss: 2.9938 - acc: 0.0590

W0120 20:01:15.829944 139667823945472 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0120 20:01:15.853226 139667823945472 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
W0120 20:01:15.860888 139667823945472 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:27: UserWarnin

Epoch 1/2
25/25 [==============================] - 323s 13s/step - loss: 2.9956 - acc: 0.0475 - val_loss: 2.9957 - val_acc: 0.0208
Epoch 2/2
25/25 [==============================] - 320s 13s/step - loss: 2.9959 - acc: 0.0500 - val_loss: 2.9956 - val_acc: 0.0000e+00


In [10]:
#!wget --no-check-certificate 'http://files.heuritech.com/weights/vgg19_weights.h5' -O vgg-16_weights.h5
#!wget -O vgg19_weights.h5 https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5

In [1]:
from keras.models import load_model
import os
img_width,img_height = 224, 224
batch_size = 16
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation


data_dir = os.path.join(os.getcwd(),'BlobStorage')
test_data_dir = os.path.join(data_dir, 'junk') # the categories need to be in folders
print(test_data_dir)

test_datagen = ImageDataGenerator(rescale=1. / 255) 

test_generator = test_datagen.flow_from_directory( 
        test_data_dir, 
        target_size=(img_width, img_height), 
        color_mode = 'rgb',
        batch_size=batch_size, 
        class_mode='categorical') 

model = load_model("model_contextobject_4classes.h5")

filenames =  test_generator.filenames
nb_samples = len(filenames)

predictions = model.predict_generator(test_generator, steps=nb_samples)

Using TensorFlow backend.
W0203 02:02:26.895384 139658233026304 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0203 02:02:26.925362 139658233026304 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0203 02:02:26.960256 139658233026304 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



/data/home/karan1234/notebooks/Karan-Paper/BlobStorage/junk
Found 846 images belonging to 3 classes.


W0203 02:02:27.260871 139658233026304 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0203 02:02:27.261746 139658233026304 deprecation_wrapper.py:119] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0203 02:02:27.270742 139658233026304 deprecation.py:506] From /anaconda/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0203 02:02:31.701847 139658233026304 deprecation_wrapper.py:119] From /anaconda/e

In [2]:
print(predictions)

[[0.0499374  0.04980759 0.04963433 ... 0.04956232 0.04979974 0.0490344 ]
 [0.04998321 0.04997195 0.05006416 ... 0.04970049 0.04967985 0.04911169]
 [0.04994777 0.05004846 0.05019908 ... 0.04966098 0.0493474  0.04908309]
 ...
 [0.04999802 0.04988209 0.04971046 ... 0.04955212 0.05015739 0.04927146]
 [0.05004388 0.0500991  0.05032448 ... 0.04976278 0.04907313 0.04900093]
 [0.04997089 0.04986463 0.04976302 ... 0.049591   0.04969763 0.0490097 ]]
